# 📰 Demo: Stock News Sentiment Analyzer

### 🎯 Goal: Extract sentiment, key events, and price catalysts from financial news articles.

### 💡 Demonstrates
- Class-based **DSPy Signatures** with typed fields  
- **LLM-as-a-judge** for evaluation  
- **GEPA optimizer** improving prompt quality

In [1]:
import dspy
from typing import Literal
import os

from dotenv import load_dotenv
load_dotenv()

# Configure LM
api_key = os.getenv("OPENAI_API_KEY")

lm = dspy.LM(model="openai/gpt-5-nano", api_key=api_key, temperature=1, max_tokens=16000)
judge_lm = dspy.LM(model="openai/gpt-5", api_key=api_key, temperature=1, max_tokens=16000)
dspy.configure(lm=lm)

print("✓ Language models configured")

✓ Language models configured


### 🧩 Train / Validation Sets

In [2]:
trainset = [
    dspy.Example(
        news="Tesla announced Q3 earnings beat with $23.4B revenue, up 8% YoY. Cybertruck production ramping faster than expected. Musk warned of margin pressure from price cuts.",
        expected_sentiment="Mixed-Positive",
        expected_catalysts="earnings beat, Cybertruck production",
        expected_risks="margin pressure"
    ).with_inputs("news"),

    dspy.Example(
        news="NVIDIA shares tumbled 7% after reports of delayed H100 chip shipments to China. Export restrictions cited as primary cause. Analysts downgraded price targets.",
        expected_sentiment="Negative",
        expected_catalysts="export restrictions, analyst downgrades",
        expected_risks="China revenue exposure"
    ).with_inputs("news"),

    dspy.Example(
        news="Apple unveiled new M3 MacBook Pro lineup with 30% performance gains. Pre-orders exceeded expectations. Services revenue hit record $22B.",
        expected_sentiment="Positive",
        expected_catalysts="M3 launch, strong pre-orders, services growth",
        expected_risks="none"
    ).with_inputs("news"),

    dspy.Example(
        news="Amazon Web Services experienced 3-hour outage affecting major customers. Stock dropped 2% on reliability concerns. AWS remains market leader with 32% share.",
        expected_sentiment="Negative",
        expected_catalysts="AWS outage",
        expected_risks="reliability concerns, customer trust"
    ).with_inputs("news"),

    dspy.Example(
        news="Meta's Reality Labs division posted $3.7B loss but Quest 3 pre-orders doubled Quest 2. Zuckerberg remains committed to metaverse investments despite investor pressure.",
        expected_sentiment="Mixed",
        expected_catalysts="Quest 3 demand",
        expected_risks="Reality Labs losses, investor pressure"
    ).with_inputs("news"),
]

valset = trainset  # For demo purposes, using same set
print(f"✓ Dataset loaded: {len(trainset)} training examples")

✓ Dataset loaded: 5 training examples


### 🔍 Define Signature / Module

In [3]:
# Define Signature
class StockNewsAnalysis(dspy.Signature):
    """Analyze financial news to extract trading signals and risk factors."""

    news: str = dspy.InputField(desc="Financial news article text")
    sentiment: Literal['Positive', 'Negative', 'Mixed', 'Mixed-Positive', 'Mixed-Negative'] = dspy.OutputField(desc="Overall market sentiment")
    key_events: str = dspy.OutputField(desc="Most important events mentioned (comma-separated)")
    price_catalysts: str = dspy.OutputField(desc="Factors that could move stock price")
    risk_factors: str = dspy.OutputField(desc="Potential risks or concerns")

# Create analyzer module
class NewsAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze = dspy.ChainOfThought(StockNewsAnalysis)

    def forward(self, news):
        return self.analyze(news=news)

print("✓ Signature and module defined")

✓ Signature and module defined


### Test Before Optimization

In [4]:
# Test Before Optimization
print("\n" + "="*60)
print("TESTING BEFORE OPTIMIZATION")
print("="*60)

analyzer = NewsAnalyzer()
test_news = trainset[0].news
result = analyzer(news=test_news)

print(f"\nNews: {test_news[:100]}...")
print(f"\nSentiment: {result.sentiment}")
print(f"Key Events: {result.key_events}")
print(f"Catalysts: {result.price_catalysts}")
print(f"Risks: {result.risk_factors}")


TESTING BEFORE OPTIMIZATION

News: Tesla announced Q3 earnings beat with $23.4B revenue, up 8% YoY. Cybertruck production ramping faste...

Sentiment: Mixed-Positive
Key Events: Q3 earnings beat; Revenue $23.4B, up 8% YoY; Cybertruck production ramp faster than expected; Musk warning of margin pressure from price cuts
Catalysts: Earnings beat and revenue growth could lift the stock; Faster Cybertruck ramp supporting future deliveries; Potential margin stabilization if price cuts do not erode profitability; Any positive guidance or demand signals in upcoming updates
Risks: Margin pressure from price cuts; Execution risk around Cybertruck ramp and production scale; Demand sustainability for EVs amid competition and macro headwinds; Potential further margin erosion if price adjustments persist


### Define Judge Metric

In [5]:
# LLM-as-a-Judge Metric
class JudgeAnalysisQuality(dspy.Signature):
    """Evaluate the quality of stock news analysis."""

    news: str = dspy.InputField()
    predicted_sentiment: str = dspy.InputField()
    predicted_catalysts: str = dspy.InputField()
    predicted_risks: str = dspy.InputField()
    expected_sentiment: str = dspy.InputField()
    expected_catalysts: str = dspy.InputField()
    expected_risks: str = dspy.InputField()
    score: float = dspy.OutputField(desc="Quality score 0-100")
    feedback: str = dspy.OutputField(desc="Detailed feedback on what's right and wrong")

class AnalysisJudge(dspy.Module):
    def __init__(self):
        super().__init__()
        self.judge = dspy.ChainOfThought(JudgeAnalysisQuality)

    def forward(self, news, predicted_sentiment, predicted_catalysts, predicted_risks,
                expected_sentiment, expected_catalysts, expected_risks):
        return self.judge(
            news=news,
            predicted_sentiment=predicted_sentiment,
            predicted_catalysts=predicted_catalysts,
            predicted_risks=predicted_risks,
            expected_sentiment=expected_sentiment,
            expected_catalysts=expected_catalysts,
            expected_risks=expected_risks
        )

# Create judge instance
judge = AnalysisJudge()
judge.set_lm(judge_lm)

print("\n✓ Judge model configured")


✓ Judge model configured


### Define GEPA Metric

In [6]:
def analysis_metric(gold, pred, trace=None, pred_name=None, pred_trace=None):
    """GEPA-compatible metric with textual feedback."""

    result = judge(
        news=gold.news,
        predicted_sentiment=pred.sentiment,
        predicted_catalysts=pred.price_catalysts,
        predicted_risks=pred.risk_factors,
        expected_sentiment=gold.expected_sentiment,
        expected_catalysts=gold.expected_catalysts,
        expected_risks=gold.expected_risks
    )

    score = float(result.score) / 100.0  # Normalize to 0-1

    return dspy.Prediction(score=score, feedback=result.feedback)

print("Metric defined")

Metric defined


### Optimize with GEPA

In [ ]:
# Optimize with GEPA
print("\n" + "="*60)
print("OPTIMIZING WITH GEPA")
print("="*60)

gepa = dspy.GEPA(
    metric=analysis_metric,
    track_stats=True,
    auto="light",
    reflection_lm=judge_lm
)

print("\nRunning GEPA optimization...")

import logging
logging.getLogger("dspy").setLevel(logging.WARNING)  # or ERROR

optimized_analyzer = gepa.compile(
    NewsAnalyzer(),
    trainset=trainset,
    valset=valset,
)


print("\n✓ Optimization complete!")


OPTIMIZING WITH GEPA

Running GEPA optimization...


GEPA Optimization:   1%|▏         | 5/400 [00:00<01:03,  6.18rollouts/s]

Average Metric: 2.72 / 3 (90.7%): 100%|██████████| 3/3 [00:00<00:00, 10.18it/s]


GEPA Optimization:   4%|▍         | 16/400 [00:02<00:59,  6.45rollouts/s]

Average Metric: 2.75 / 3 (91.7%): 100%|██████████| 3/3 [00:00<00:00,  9.23it/s]


...

Average Metric: 2.87 / 3 (95.7%): 100%|██████████| 3/3 [00:01<00:00,  1.74it/s]


GEPA Optimization:  41%|████      | 163/400 [03:40<27:00,  6.84s/rollouts]

Average Metric: 2.87 / 3 (95.7%): 100%|██████████| 3/3 [00:01<00:00,  2.19it/s]


GEPA Optimization:  42%|████▏     | 169/400 [05:13<35:09,  9.13s/rollouts]

Average Metric: 2.75 / 3 (91.7%): 100%|██████████| 3/3 [00:00<00:00,  8.37it/s]


### 🔍 Test After Optimization

In [ ]:
# Test After Optimization
print("\n" + "="*60)
print("TESTING AFTER OPTIMIZATION")
print("="*60)

result_optimized = optimized_analyzer(news=test_news)

print(f"\nNews: {test_news[:100]}...")
print(f"\nSentiment: {result_optimized.sentiment}")
print(f"Key Events: {result_optimized.key_events}")
print(f"Catalysts: {result_optimized.price_catalysts}")
print(f"Risks: {result_optimized.risk_factors}")

### Compare Performance

In [ ]:
# Compare Performance
print("\n" + "="*60)
print("PERFORMANCE COMPARISON")
print("="*60)


def evaluate(analyzer, dataset):
    scores = []
    for example in dataset:
        pred = analyzer(news=example.news)
        result = analysis_metric(gold=example, pred=pred)
        scores.append(result.score)
    return sum(scores) / len(scores)


print("\nEvaluating performance...")
before_score = evaluate(NewsAnalyzer(), valset)
after_score = evaluate(optimized_analyzer, valset)

print(f"\nAverage Score Before: {before_score:.2f}")
print(f"Average Score After:  {after_score:.2f}")
print(f"Improvement: {((after_score - before_score) / before_score * 100):.1f}%")